<u>Imports</u>

In [10]:
# Standard
import pandas as pd
import numpy as np
import requests
from scipy.stats import randint

# Scrapping
from selenium import webdriver # Version == 4.22.0
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions
from selenium.common.exceptions import TimeoutException

# Text processing
import re

# Dataset preparation
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.impute import SimpleImputer

# Modelling
from sklearn.model_selection import train_test_split, cross_validate, GridSearchCV, RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report
from xgboost import XGBClassifier

# Visualisation
import matplotlib.pyplot as plt

<u>Scrapping</u>

In [25]:
url = 'https://www.windguru.cz/112'
num_prev = 100 # Number of observations to be collected from Windguru

In [137]:
class Scraper:

    def __init__(self, url):
        self.url = url
        self.driver = webdriver.Chrome()
        self.driver.get(self.url)
        
    def scrape(self, num_prev):

        """
        Scrapes forecast data for a specified number of observations (1 observation = 2-hour period)
        Args:
        * num_prev: Number of forecast periods to scrape.
        Returns:
        * A pandas DataFrame containing the scraped forecast data:
            - Date & hour of estimate
            - Wind and gust speed and direction
            - Swell height, period and direction
        """ 

        # Wait for the browsed page before scraping
        try:
            myElem = WebDriverWait(self.driver, 5).until(expected_conditions.presence_of_element_located((By.XPATH, '//*[@id="tabid_0_0_dates"]/td[1]')))
        except TimeoutException:
            None
        
        forecast = {}

        parse_number = lambda x: int(''.join([l for l in str(x) if l.isdigit()]))

        # Extract datetime
        temp_list = []
        for i in range(1, num_prev + 1):
            try:
                value = self.driver.find_element(By.XPATH, f'//*[@id="tabid_0_0_dates"]/td[{i}]')
                temp_list.append(value.text)
            except Exception as e:
                temp_list.append(pd.NA)
        forecast['date'] = temp_list

        # Extract numeric figures 
        for name in ['tabid_0_0_WINDSPD','tabid_0_0_GUST','tabid_0_0_HTSGW', 'tabid_0_0_PERPW'] :
            temp_list = []
            for i in range(1, num_prev + 1):
                try:
                    value = self.driver.find_element(By.XPATH, f'//*[@id="{name}"]/td[{i}]')
                    numeric_value = float(value.text.strip())  # Convert text to float
                    temp_list.append(numeric_value)
                except Exception as e:
                    temp_list.append(pd.NA)
            forecast[name] = temp_list

        # Extract angles        
        for name in ['tabid_0_0_SMER','tabid_0_0_DIRPW']:
            temp_list = []
            for i in range(1, num_prev + 1):
                try:
                    value = self.driver.find_element(By.XPATH, f'//*[@id="{name}"]/td[{i}]/span')
                    numeric_value = parse_number(value.get_attribute('title'))
                    temp_list.append(numeric_value)
                except Exception as e:
                    temp_list.append(pd.NA)
            forecast[name] = temp_list
        
        forecast_df=pd.DataFrame(forecast)
        
        # Formatting 
        forecast_df.dropna(inplace=True)
        forecast_df.columns = ['date','wind_speed','gust_speed','swell_height','swell_period','wind_dir','swell_dir']
        forecast_df['wind_speed'] = forecast_df[['wind_speed', 'gust_speed']].mean(axis=1)
        forecast_df = forecast_df.drop(columns=['gust_speed'])
        forecast_df[['wind_speed','swell_period']] = forecast_df[['wind_speed','swell_period']].astype(int)
        
        return forecast_df

In [138]:
scraper = Scraper(url)
forecast_df = scraper.scrape(num_prev = num_prev)

In [139]:
forecast_df

,date,wind_speed,swell_height,swell_period,wind_dir,swell_dir
0,Mo\n1.\n11h,14,0.6,4,319,317
1,Mo\n1.\n13h,12,0.6,4,313,315
2,Mo\n1.\n15h,12,0.5,4,309,310
3,Mo\n1.\n17h,15,0.6,3,303,304
4,Mo\n1.\n19h,16,0.8,4,306,306
...,...,...,...,...,...,...
95,Su\n14.\n11h,7,0.3,2,37,37
96,Su\n14.\n14h,9,0.4,3,21,26
97,Su\n14.\n17h,11,0.5,3,21,30
98,Su\n14.\n20h,13,0.5,3,17,27


<u>Dataset preparation</u>

In [ ]:
dataset = pd.read_excel('../data/dataset_rochebonne.xlsx')
dataset = dataset.sample(frac=1, random_state=42).reset_index(drop=True)
dataset.head()

In [ ]:
dataset.info()

In [ ]:
dataset.describe()

In [ ]:
# Plot distribution

fig, axes = plt.subplots(nrows=2, ncols=3, figsize=(15, 6))
columns = ['wind_speed', 'wind_dir', 'swell_height', 'swell_period', 'swell_dir', 'note']
colors = ['skyblue', 'royalblue', 'dodgerblue', 'deepskyblue', 'steelblue', 'cornflowerblue']

for ax, col, color in zip(axes.flatten(), columns, colors):
    ax.hist(dataset[col], bins=30, color=color, edgecolor='black', alpha=0.7)
    ax.set_title(col)
    ax.grid(True, linestyle='--', alpha=0.6)

plt.tight_layout()
plt.show()

In [ ]:
dataset.note.value_counts()

In [ ]:
# Split X and y

y = dataset['note']
X = dataset.drop(columns=['note'])

In [ ]:
# Split test and train sets

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Scale data 

minmax_scaler = MinMaxScaler()
standard_scaler = StandardScaler()

X_train_scaled = X_train.copy()
X_test_scaled = X_test.copy()

X_train_scaled[['wind_speed', 'wind_dir', 'swell_dir']] = minmax_scaler.fit_transform(X_train[['wind_speed', 'wind_dir', 'swell_dir']])
X_test_scaled[['wind_speed', 'wind_dir', 'swell_dir']] = minmax_scaler.transform(X_test[['wind_speed', 'wind_dir', 'swell_dir']])

X_train_scaled[['swell_height', 'swell_period']] = standard_scaler.fit_transform(X_train[['swell_height', 'swell_period']])
X_test_scaled[['swell_height', 'swell_period']] = standard_scaler.transform(X_test[['swell_height', 'swell_period']])

In [ ]:
X_train_scaled.describe()

In [ ]:
X_test_scaled.describe()

<u>Modelling</u>

Comparison of the following:
- Logistic Regression,
- Decision Tree,
- Random Forest,
- KNN,
- XGBoost,
- Naïve Bayes,
- Support Vector Machines

In [ ]:
accuracy = {} # Store the performance of each model

1- Logistic Regression

In [ ]:
logistic_model = LogisticRegression(solver='newton-cg')

In [ ]:
cv_results_log = cross_validate(logistic_model, X_train_scaled, y_train, scoring = 'accuracy', cv=5)
cv_results_log

In [ ]:
accuracy['LogisticRegression'] = cv_results_log['test_score'].mean().round(2)

In [ ]:
logistic_model.fit(X_train_scaled,y_train)

In [ ]:
logistic_model.score(X_test_scaled,y_test)

2- DecisionTreeClassifier

In [ ]:
decision_tree = DecisionTreeClassifier()

In [ ]:
cv_results_dt = cross_validate(decision_tree, X_train, y_train, scoring = 'accuracy', cv=5)
cv_results_dt

In [ ]:
print(f'Average accuracy before RandomizedSearch: {cv_results_dt["test_score"].mean().round(3)}')

In [ ]:
accuracy['DecisionTree'] = cv_results_dt['test_score'].mean().round(2)

In [ ]:
decision_tree.fit(X_train,y_train)

In [ ]:
decision_tree.score(X_test,y_test)

3- Random Forest

In [ ]:
forest = RandomForestClassifier()

In [ ]:
cv_results = cross_validate(forest, X_train, y_train, scoring = 'accuracy', cv=5)
cv_results

In [ ]:
print(f'Average accuracy before RandomizedSearch: {cv_results["test_score"].mean().round(3)}')

In [ ]:
forest.get_params()

In [ ]:
# Perform randomized search for hyperparameter tuning

param_grid_forest = {'n_estimators': [int(x) for x in np.linspace(start = 10, stop = 120, num = 10)], # Number of trees
                'max_features': ['log2', 'sqrt', None], # Number of features at every split
                'max_depth': [None,1,2,4,6,8,10], # Maximum number of levels in tree
                'min_samples_split': [2,3,4,5], # Minimum number of samples to split a node
                'min_samples_leaf': [1, 2], # Minimum number of samples at each leaf node
                'bootstrap': [True, False]} # Method of selecting samples for growing each tree

forest_search = RandomizedSearchCV(estimator = forest, 
                                   param_distributions = param_grid_forest, 
                                   scoring='accuracy', 
                                   n_iter = 100, 
                                   cv = 5, 
                                   verbose=1, 
                                   random_state=42, 
                                   n_jobs = -1)

forest_search.fit(X_train, y_train)

In [ ]:
forest_best = forest_search.best_estimator_

In [ ]:
cv_results_optimized = cross_validate(forest_best, X_train, y_train, scoring = 'accuracy', cv=5)
cv_results_optimized

In [ ]:
forest_best.fit(X_train,y_train)

In [ ]:
y_pred = forest_best.predict(X_test)

In [ ]:
print(f'Average accuracy after RandomizedSearch: {cv_results_optimized["test_score"].mean().round(2)}')

In [ ]:
target_names = ['rating 0', 'rating 1', 'rating 2', 'rating 3']
print(classification_report(y_test, y_pred, target_names=target_names))

In [ ]:
accuracy['RandomForest'] = cv_results_optimized["test_score"].mean().round(2)

In [ ]:
forest_best.fit(X_train,y_train)

In [ ]:
forest_best.score(X_test,y_test)

4- KNeighbors

In [ ]:
knn = KNeighborsClassifier()

In [ ]:
knn.get_params()

In [ ]:
cv_results_knn = cross_validate(knn, X_train_scaled, y_train, scoring = 'accuracy', cv=5)
cv_results_knn

In [ ]:
print(f'Average accuracy before RandomizedSearch: {cv_results_knn["test_score"].mean().round(3)}')

In [ ]:
# Grid search for hyperparameter tuning

param_grid_knn = {'n_neighbors': [1, 3, 5, 7, 9, 11, 13], # Number of neighbours
              'weights': ['uniform', 'distance'], # Weight function used / uniform weights or relative to their distance
              'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'], # Algorithm used to compute the nearest neighbors
              'p': [1, 2] # 1 = manhattan_distance, 2 = euclidean_distance
             }

knn_search = GridSearchCV(estimator = knn, 
                            param_grid = param_grid_knn, 
                            scoring='accuracy', 
                            cv = 5, 
                            verbose=1, 
                            n_jobs = -1)

knn_search.fit(X_train_scaled, y_train)

In [ ]:
knn_best = knn_search.best_estimator_

In [ ]:
cv_results_knn_best = cross_validate(knn_best, X_train_scaled, y_train, scoring = 'accuracy', cv=5)
cv_results_knn_best

In [ ]:
print(f'Average accuracy after RandomizedSearch: {cv_results_knn_best["test_score"].mean().round(3)}')

In [ ]:
knn.fit(X_train_scaled,y_train)

In [ ]:
knn.score(X_test_scaled,y_test)

5- XGBoost

In [ ]:
xgb_model = XGBClassifier(objective='multi:softprob')

In [ ]:
cv_results_xgb = cross_validate(xgb_model, X_train_scaled, y_train, scoring = 'accuracy', cv=5)
cv_results_xgb

In [ ]:
print(f'Average accuracy before RandomizedSearch: {cv_results_xgb["test_score"].mean().round(3)}')

In [ ]:
# Perform randomized search for hyperparameter tuning

xgb_params = {
            'n_estimators': [int(x) for x in np.linspace(start = 10, stop = 120, num = 10)],
            'max_depth' : [None, 1, 2, 4, 6, 8, 10],
            'min_child_weight': [None, 1, 5, 10]
        }

xgb_search = GridSearchCV(estimator = xgb_model, 
                                   param_grid = xgb_params, 
                                   scoring='accuracy', 
                                   cv = 5, 
                                   verbose=1, 
                                   n_jobs = -1)

xgb_search.fit(X_train_scaled, y_train)

In [ ]:
xgb_best = xgb_search.best_estimator_

In [ ]:
cv_results_xgb_best = cross_validate(xgb_best, X_train_scaled, y_train, scoring = 'accuracy', cv=5)

In [ ]:
xgb_best.score(X_test_scaled,y_test)

In [ ]:
print(f'Average accuracy after RandomizedSearch: {cv_results_xgb_best["test_score"].mean().round(3)}')

<u>Forecasts</u>

In [ ]:
X_new_values = forecast_df[['wind_speed', 'wind_dir', 'swell_height', 'swell_period', 'swell_dir']]

In [ ]:
y_new_values = forest_best.predict(X_new_values)

In [ ]:
X_new_values

In [ ]:
X_new_values_scaled = X_new_values.copy()

In [ ]:
X_new_values_scaled[['wind_speed', 'wind_dir', 'swell_dir']] = minmax_scaler.transform(X_new_values[['wind_speed', 'wind_dir', 'swell_dir']])
X_new_values_scaled[['swell_height', 'swell_period']] = standard_scaler.transform(X_new_values[['swell_height', 'swell_period']])

In [ ]:
y_new_values = xgb_best.predict(X_new_values_scaled)

In [ ]:
y_new_values

In [ ]:
forecast_df['note'] = y_new_values